# 1

有关树的操作，基本递归比较好使。

由于可以从任意节点开始，\
所以应该传递一个列表，\
1.列表需要记录到这个节点之前的累积和；\
2.然后就是如果从当前节点开始，也要加入列表。\
最后判断是否满足条件。

![](../imgs/437.svg)

> 为了节省空间，可以在递归前压入，返回后弹出。\
> 由于会对每个元素都进行操作，弹出后还需要对每个元素减去压入的值。

此处还是直接复制列表吧，方便。

In [5]:
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right


class Solution:
    def pathSum(self, root: TreeNode, targetSum: int) -> int:
        self.ans = 0
        def dfs(node, target_sum, record):
            record.append(0)
            record = list(map(lambda x: x+node.val, record))
            self.ans += record.count(target_sum)
            if node.left:
                dfs(node.left, target_sum, record.copy())
            if node.right:
                dfs(node.right, target_sum, record.copy())
        if root:
            dfs(root, targetSum, [])
        return self.ans


if __name__ == '__main__':
    root = TreeNode(10)
    root.left, root.right = TreeNode(5), TreeNode(-3)
    root.left.left, root.left.right, root.right.right = TreeNode(3), TreeNode(2), TreeNode(11)
    root.left.left.left, root.left.left.right, root.left.right.right = TreeNode(3), TreeNode(-2), TreeNode(1)
    print(Solution().pathSum(root, 8))

    print(Solution().pathSum(None, 0))

    root = TreeNode(0)
    root.left, root.right = TreeNode(1), TreeNode(1)
    print(Solution().pathSum(root, 1))

3
0
4


126/126 cases passed (220 ms)\
Your runtime beats 48.37 % of python3 submissions\
Your memory usage beats 5.02 % of python3 submissions (39.9 MB)

time: O(nlogn)

space: O(2^logn * logn)

> 每个节点都对记录的所有元素都进行了一次加法运算。\
> 二叉树有logn层，叶子节点有2^logn个列表记录，每个记录最多有logn个元素。\
> 如果复用记录的空间的话，可以降低空间复杂度到O(logn)，对于调用开辟的调用栈，空间复杂度应该为O(n)。

# 题解1

思路一样，\
但设计了2个函数的递归，实现了从不同节点开始。

这样的好处是不需要用列表记录，空间开销变小，但栈的递归变多了。

In [14]:
class Solution:
    def pathSum(self, root: TreeNode, targetSum: int) -> int:
        def rootSum(root, targetSum):
            if root is None:
                return 0

            ret = 0
            if root.val == targetSum:
                ret += 1

            ret += rootSum(root.left, targetSum - root.val)
            ret += rootSum(root.right, targetSum - root.val)
            return ret
        
        if root is None:
            return 0
            
        ret = rootSum(root, targetSum)
        ret += self.pathSum(root.left, targetSum)
        ret += self.pathSum(root.right, targetSum)
        return ret

time: O(n^2)

space: O(n)

> 空间主要是函数调用，栈的开辟。

# 题解2

题解方法比较巧妙，但不具普适性。

方法被称为前缀和，图解比较容易理解。

![](../imgs/437-ans.svg)

求出从根开始到每个节点的和，\
将其记录到字典中，\
最后，查找字典中节点之间的差等于目标值即可。

（例如，图中，有三个节点值为18，它们减去节点值10，就可以的到目标值。）

In [15]:
import collections

class Solution:
    def pathSum(self, root: TreeNode, targetSum: int) -> int:
        prefix = collections.defaultdict(int)
        prefix[0] = 1

        def dfs(root, curr):
            if not root:
                return 0
            
            ret = 0
            curr += root.val
            ret += prefix[curr - targetSum]
            prefix[curr] += 1
            ret += dfs(root.left, curr)
            ret += dfs(root.right, curr)
            prefix[curr] -= 1

            return ret

        return dfs(root, 0)

time: O(n)

space: O(n)